In [ ]:
import pandas as pd
import arrow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sys
%matplotlib inline
import emission.storage.timeseries.abstract_timeseries as esta

import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow
import emission.core.get_database as edb
from emission.core.wrapper.user import User

In [ ]:
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow

Get the dataframe that we need

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get("2018-3-28").timestamp, arrow.get("2018-6-6").timestamp)
server_api_calls_df = agts.get_data_df("stats/server_api_time", time_query=sep_dec_tq_data_ts)

In [ ]:
agts = estag.AggregateTimeSeries()
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)
profile_create_api_calls_df = server_api_calls_df[server_api_calls_df.name == 'POST_/profile/notification_open']
client_nav_events_df[client_nav_events_df.name.str.contains('state')]
client_nav_events_df.head()

In [ ]:
users = edb.get_uuid_db().find()
information_uuids = []
emotion_uuids = []
control_uuids = []
mode_mapping = {0: "Unkown Type", 1: "Walking", 2: "Bicycling", 3: "Bus", 4: "Train", 5: "Car", 6: "Air"}
for user_id in users:
    ts = esta.TimeSeries.get_time_series(user_id)
    last_period_tq = estt.TimeQuery("data.start_ts",
                            arrow.get("2018-3-28").timestamp, # start of range
                            arrow.get("2018-6-6").timestamp)  # end of range
    cs_df = ts.get_data_df("analysis/inferred_section", time_query=last_period_tq)
    if cs_df.shape[0] > 0:
        try:
            client = edb.get_profile_db().find_one({"user_id": user_id})['client']
            if client == 'urap-2017-information':
                information_uuids.append(user_id)
            elif client == 'urap-2017-emotion':
                emotion_uuids.append(user_id)
            elif client == 'urap-2017-control':
                control_uuids.append(user_id)
        except:
            continue
print(len(information_uuids) + len(emotion_uuids) + len(control_uuids))

In [ ]:
information_users = pd.DataFrame([i for i in information_uuids]).rename(columns={0: 'user_id'})
emotion_users = pd.DataFrame([i for i in emotion_uuids]).rename(columns={0: 'user_id'})
control_users = pd.DataFrame([i for i in control_uuids]).rename(columns={0: 'user_id'})
information_users.head()

In [ ]:
edb.get_profile_db()

In [ ]:
notifications_df = client_nav_events_df[client_nav_events_df['name'] == 'notification_open']

In [ ]:
notifications_df = notifications_df.fillna(1)
user_notifications = notifications_df.groupby('user_id').sum()
user_notifications.head()

In [ ]:
information_notifications = pd.merge(
    information_users, user_notifications, how='left', left_on='user_id', right_index=True).fillna(0).rename(columns={'reading': 'num_opened'})

emotion_notifications = pd.merge(
    emotion_users, user_notifications, how='left', left_on='user_id', right_index=True).fillna(0).rename(columns={'reading': 'num_opened'})

control_notifications = pd.merge(
    control_users, user_notifications, how='left', left_on='user_id', right_index=True).fillna(0).rename(columns={'reading': 'num_opened'})

information_notifications

## Average notifcations opened per app
This is a very simplistic measurement. We need to normalize by the number of total notifactions, because this is different for each user and version.

We dont have that :(

In [ ]:
information_avg = np.mean(information_notifications['num_opened'])
information_med = np.median(information_notifications['num_opened'])
informtaiton_sd = np.std(information_notifications['num_opened'])
information_avg, information_med, informtaiton_sd

In [ ]:
emotion_avg = np.mean(emotion_notifications['num_opened'])
emotion_med = np.median(emotion_notifications['num_opened'])
emotion_sd = np.std(emotion_notifications['num_opened'])
emotion_avg, emotion_med, emotion_sd

In [ ]:
info_counts = information_notifications['num_opened']
plt.hist(info_counts);
plt.title('Information: Distribution of Notifications Opened');

In [ ]:
emotion_counts = emotion_notifications['num_opened']
plt.hist(emotion_counts, bins = np.arange(35));
plt.title('Emotion: Distribution of Notifications Opened');